## USFSA Results Scraping

### Part 1: Scraping the Webpages

#### Imports

In [26]:
import pandas as pd
import requests
import bs4

#### Create a get request for the main URL

In [2]:
urls = {'2023-nationals': 
        'https://ijs.usfigureskating.org/leaderboard/nonqual_results/2023/32002/index.html',
        
        '2023-ride-the-tide': 
        'https://ijs.usfigureskating.org/leaderboard/nonqual_results/2022/32006/index.html',
        
        '2023-golden-bear-skate':
        'https://ijs.usfigureskating.org/leaderboard/nonqual_results/2022/32004/index.html',
        
        '2022-pioneer-open':
        'https://ijs.usfigureskating.org/leaderboard/nonqual_results/2022/32007/index.html',
       }

In [3]:
# Change url to the competition of choice from the above dictionary
main_url = urls['2023-ride-the-tide']
request  = requests.get(main_url)

In [4]:
# Global Variables
soup   = bs4.BeautifulSoup(request.text)
events = soup.find_all("td", attrs = {"rowspan": 1})
links  = soup.find_all("td", attrs = {"class": "cm rb"})

In [5]:
baseurl = main_url.replace("index.html", "")

# Helper function to extract links
def extract_link(x):
    return x.find("a")["href"]

# Extract the url ends for each webpage
ends = list(map(extract_link, links))

# Request urls for each webpage
webpages = [baseurl + i for i in ends]
    

#### Function for processing one results page

In [6]:
# Function to parse each results page
def parse_results(html, team = False):
    
    '''
    Takes an html text object containing the results of
    one group
    
    :params html: html text
    :returns: A DataFrame containing the place and university
    for each start
    '''
    
    # Create a soup object and extract the rows of the results table
    soup = bs4.BeautifulSoup(html.text)
    res  = soup.find_all("td", attrs = {"colspan":1})
    rows = soup.find_all("tr")
    rows = [x for x in rows if (len(x.find_all("td")) == 9 or len(x.find_all("td")) == 7)]
    
    # Extract the University names from each page
    out = []
    for i, x in enumerate(res):
        if team:
            uni = x.text
        else:
            uni = x.text.split(", ")[-1]
        out.append([rows[i].find("td").text, uni, rows[i].find_all("td")[-1].text])
        
    return pd.DataFrame(out, columns = ["Place", "College", "Tie"])

#### Loop through each page and extract the data

In [7]:
DFS = []
for i, x in enumerate(webpages):
    temp = requests.get(x)
    try:
        data = parse_results(temp)
        data = data.loc[~data["Tie"].str.contains("Withdraw")]
        DFS.append(data)
    except:
        pass   

### Part 2: Calculating the points awarded for each event

In [8]:
lookup = {24: [12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
          23: [12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
          22: [12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
          21: [12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
          20: [12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1],
          19: [12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1],
          18: [12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 1, 1, 1, 1, 1, 1],
          17: [12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 1, 1, 1, 1, 1],
          16: [12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 1, 1, 1, 1],
          15: [12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 1, 1, 1],
          14: [12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 1, 1],
          13: [12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 1],
          12: [12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1],
          11: [12, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1],
          10: [12, 10, 8, 7, 6, 5, 4, 3, 2, 1],
          9: [12, 10, 8, 6, 5, 4, 3, 2, 1],
          8: [12, 10, 8, 6, 4, 3, 2, 1],
          7: [12, 10, 8, 6, 4, 2, 1],
          6: [12, 10, 8, 6, 4, 2],
          5: [10, 8, 6, 4, 2],
          4: [8, 6, 4, 2],
          3: [6, 4, 2],
          2: [6, 4],
          1: [6]
         }

In [9]:
OUT = []
for i, x in enumerate(DFS):
    num = len(x)
    
    # Assign a number of points to each column
    x = x.assign(points = lookup[num])

    # Handle ties
    temp = x.groupby("Place")["points"].transform(lambda x: x.mean())
    x = x.assign(points = temp)
    
    # Handle championship event edge case
    if "Championship" in events[i].text or "International" in events[i].text:
        x["points"] = x["points"] + 2
    
    OUT.append(x)

In [10]:
FULL = pd.concat(OUT)

# Eliminate the error where it says 'U' instead of 'University'
# Keep in mind there are still a few errors with the names of the colleges
# that I have not fixed yet
import re
FULL['College'] = FULL['College'].apply(lambda x: re.sub('^U ', 'University ', x))
FULL['College'] = FULL['College'].apply(lambda x: re.sub(' U$', ' University', x))

#### Team Standings

In [11]:
A = FULL.groupby("College")["points"].sum().sort_values(ascending = False)
A

College
University of Denver                 245.0
University of CA - Berkeley          234.0
University of CA - San Diego         213.5
University of CO - Boulder           203.0
University of CA - Los Angeles       197.0
University of Washington             141.0
Arizona State University             131.0
Stanford University                  104.5
Colorado State Univer...              80.0
Utah State University                 73.0
Colorado College                      60.5
University of CO - Colorado Sp...     58.0
Western Washington Un...              58.0
University of Souther...              57.5
University of CA - Davis              45.5
University of CA - Santa Barbara      42.0
University of Norther...              21.5
University of CA - Irvine             21.0
Colorado State University             19.0
University of CO - Colo. Springs      18.0
University of Southern California     15.0
University of Wyoming                 14.0
Western Washington University         12.0
Uni

#### Number of Starts per Team

In [12]:
B = FULL.groupby("College").count()["Place"].sort_values(ascending=False)
B

College
University of CO - Boulder           30
University of CA - Berkeley          30
University of Denver                 30
University of CA - San Diego         30
University of CA - Los Angeles       28
Arizona State University             25
Stanford University                  22
Colorado State Univer...             20
University of Washington             19
Colorado College                     14
University of CO - Colorado Sp...    13
Western Washington Un...             13
University of Souther...             12
Utah State University                11
University of CA - Davis             10
University of CA - Santa Barbara      8
University of CA - Irvine             5
University of Norther...              5
University of Wyoming                 5
University of Southern California     3
Colorado State University             3
University of CO - Colo. Springs      2
Univesity of Wyoming                  1
Western Washington University         1
Name: Place, dtype: int64

#### Points per start ratio

In [13]:
C = pd.merge(A.to_frame(), B.to_frame(), left_index=True, right_index=True)
C.columns = ["Points", "Number of Starts"]

C["Ratio"] = C["Points"] / C["Number of Starts"]
C.sort_values(by="Ratio", ascending = False)

,Points,Number of Starts,Ratio
College,,,
Western Washington University,12.0,1,12.000000
University of CO - Colo. Springs,18.0,2,9.000000
University of Denver,245.0,30,8.166667
Univesity of Wyoming,8.0,1,8.000000
University of CA - Berkeley,234.0,30,7.800000
University of Washington,141.0,19,7.421053
University of CA - San Diego,213.5,30,7.116667
University of CA - Los Angeles,197.0,28,7.035714
University of CO - Boulder,203.0,30,6.766667


#### Number of Starts

In [14]:
C["Number of Starts"].sum()

340

#### To get a general count of how many of each type of official each competition has

In [15]:
import numpy as np

In [16]:
DFS = []
for i, x in enumerate(webpages):
    temp = requests.get(x)
    

In [17]:
def process_judges(text):
    soup = bs4.BeautifulSoup(text)
    tabs = np.array(list(map(lambda x: x.text, soup.find_all('td'))))
    offs = tabs[np.where(tabs == 'Judge 1')[0][0]:][1::3][:-1]
    return list(offs)

In [18]:
def process_referees(text):
    soup = bs4.BeautifulSoup(text)
    tabs = tabs = np.array(list(map(lambda x: x.text, soup.find_all('td'))))
    refs = tabs[np.where(tabs == 'Referee')[0][0] + 1]
    return [refs]

In [19]:
def process_accountants(text):
    soup = bs4.BeautifulSoup(text)
    tabs = tabs = np.array(list(map(lambda x: x.text, soup.find_all('td'))))
    accs = tabs[np.where(tabs == 'Accountant')[0][0] + 1]
    return [accs]

In [20]:
JUDGES = []
for i, x in enumerate(webpages):
    temp = requests.get(x)
    jlst = process_judges(temp.text)
    JUDGES += jlst

In [21]:
REFEREES = []
for i, x in enumerate(webpages):
    temp = requests.get(x)
    rlst = process_referees(temp.text)
    REFEREES += rlst

In [22]:
ACCOUNTANTS = []
for i, x in enumerate(webpages):
    temp = requests.get(x)
    alst = process_accountants(temp.text)
    ACCOUNTANTS += alst

In [23]:
pd.Series(JUDGES).unique()

array(['Heather Correa', 'Yuu Ohno', 'Rochelle Revor', 'Kim Borzilleri',
       'Jill Montgomery', 'Karen Silverman', 'George Rossano*',
       'Doug Williams', 'David Hold', 'Stacy Brinkman',
       'Stefanie Mathewson', 'Carylyn Landt', 'Michael Ricigliano',
       'Caroline Guttman'], dtype=object)

In [24]:
pd.Series(REFEREES).unique()

array(['George Rossano*', 'Doug Williams', 'Caroline Guttman',
       'Stefanie Mathewson', 'David Hold', 'Michael Ricigliano',
       'Carylyn Landt'], dtype=object)

In [25]:
pd.Series(ACCOUNTANTS).unique()

array(['Jayne Rendel*', 'Suzie Whitehead', 'Elda Stillwell'], dtype=object)